In [ ]:
from langchain import hub
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.utilities import SQLDatabase
from langchain_openai import ChatOpenAI
from sqlalchemy import create_engine
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.message import MessagesState
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain_core.tools import tool, InjectedToolCallId
from langgraph.prebuilt import InjectedState, create_react_agent
from langgraph.types import Command
from typing import TypedDict, Annotated, List, Dict, Any, Optional, Literal
import json
import os
from datetime import datetime
from pydantic import BaseModel, Field
from src.models.database import get_mongo_memory
from src.tools.custom_toolkit import CustomToolkit
from src.utils.chart_utils import get_supported_charts
try:
    from explainer import Explainer
    from nodes.planner_node import PlannerNode
except ImportError:
    from .explainer import Explainer
    from ..nodes.planner_node import PlannerNode


In [ ]:
# Use the same state as the original explainable agent
class ExplainableAgentState(MessagesState):
    query: str
    plan: str
    steps: List[Dict[str, Any]]
    step_counter: int
    human_comment: Optional[str]
    status: Literal["approved", "feedback", "cancelled"]
    assistant_response: str
    use_planning: bool = True  # Planning preference from API
    use_explainer: bool = True  # Whether to use explainer node
    response_type: Optional[Literal["answer", "replan", "cancel"]] = None  # Type of response from planner
    agent_type: str = "data_exploration_agent"  # Which specialized agent to use
    routing_reason: str = ""  # Why this agent was chosen
    visualizations: Optional[List[Dict[str, Any]]] = []

print("✅ State definition ready!")


In [ ]:
# Import planner node from existing codebase
from src.nodes.planner_node import PlannerNode

print("✅ Planner node imported!")


In [ ]:
# Complete ExplainableAgent class (extracted from original)
class ExplainableAgent:
    """Data Exploration Agent - Specialized for SQL database queries and data analysis with explanations"""
    
    def __init__(self, llm, db_path: str, logs_dir: str = None, mongo_memory=None):
        self.llm = llm
        self.db_path = db_path
        self.engine = create_engine(f'sqlite:///{db_path}')
        self.db = SQLDatabase(self.engine)
        self.toolkit = SQLDatabaseToolkit(db=self.db, llm=self.llm)
        self.sql_tools = self.toolkit.get_tools()
        
        # Create custom toolkit with LLM
        self.custom_toolkit = CustomToolkit(llm=self.llm)
        self.custom_tools = self.custom_toolkit.get_tools()
        
        # Combine all tools
        self.tools = self.sql_tools + self.custom_tools
        self.explainer = Explainer(llm)
        self.planner = PlannerNode(llm, self.tools)
        self.logs_dir = logs_dir or os.path.join(os.path.dirname(os.path.dirname(os.path.dirname(__file__))), "logs")
        os.makedirs(self.logs_dir, exist_ok=True)
        self.mongo_memory = mongo_memory
        
        # Create handoff tools for the assistant
        self.create_handoff_tools()
        
        # Create assistant as a react agent with transfer tools
        base_assistant_agent = create_react_agent(
            model=llm,
            tools=[self.transfer_to_data_exploration],  # Add transfer_to_explainer_agent when ready
            prompt=(
                "You are an assistant that routes tasks to specialized agents.\n\n"
                "AVAILABLE AGENTS:\n"
                "- data_exploration_agent: Handles database queries and visualizations, SQL analysis, and data exploration\n"
                "  Use this for: SQL queries, database analysis, table inspection, data queries, schema questions\n\n"
                "FUTURE AGENTS (planned):\n"
                "- explainer_agent: Explains concepts, processes, code, or any topic to users\n"
                "  Will be used for: explanations, tutorials, concept clarification, how-to questions\n\n"
                "INSTRUCTIONS:\n"
                "- Analyze the user's request and determine which specialized agent should handle it\n"
                "- For database/SQL related queries and visualizations, use transfer_to_data_exploration\n"
                "- Be helpful and direct in your routing decisions\n"
                "- IMPORTANT: Only route to agents when you receive a NEW user message, not for agent responses\n"
                "- CRITICAL: Make only ONE tool call per user message. Pass the full task in a single call.\n"
                "- The specialized agent will handle all aspects of the request (multiple charts, queries, etc.)\n"
                "- Example: If user asks for '3 different charts', call the transfer tool ONCE with the full request\n"
            ),
            name="assistant"
        )
        
        # Store use_planning value for tools to access
        self._use_planning = None
        self._use_explainer = None
        
        def assistant_agent(state):
            use_planning = state.get("use_planning", True)
            use_explainer = state.get("use_explainer", True)
            agent_type = state.get("agent_type", "data_exploration_agent")
            query = state.get("query", "")
            
            # Store use_planning value for tools to access
            self._use_planning = use_planning
            self._use_explainer = use_explainer
            result = base_assistant_agent.invoke(state)
            
            if isinstance(result, dict):
                result["use_planning"] = use_planning
                result["use_explainer"] = use_explainer
                result["agent_type"] = agent_type
                result["query"] = query
            
            return result
        
        self.assistant_agent = assistant_agent
        
        # Create the graph
        self.graph = self.create_graph()

print("✅ ExplainableAgent class defined!")


In [ ]:
# Add the handoff tools method to the ExplainableAgent class
def create_handoff_tools(self):
    """Create handoff tools for the assistant to transfer to specialized agents"""
    
    @tool("transfer_to_data_exploration", description="Transfer database and SQL queries to the data exploration agent")
    def transfer_to_data_exploration(
        state: Annotated[Dict[str, Any], InjectedState],
        tool_call_id: Annotated[str, InjectedToolCallId],
        task_description: str = ""
    ) -> Command:
        """Transfer to data exploration agent"""
        
        tool_message = {
            "role": "tool",
            "content": f"Transferring to data exploration agent: {task_description}",
            "name": "transfer_to_data_exploration",
            "tool_call_id": tool_call_id,
        }
        
        # Extract query from messages if not in state
        query = state.get("query", "")
        if not query and "messages" in state and state["messages"]:
            # Get the first human message as the query
            for msg in state["messages"]:
                if hasattr(msg, 'content') and hasattr(msg, '__class__') and 'HumanMessage' in str(msg.__class__):
                    query = msg.content
                    break
        
        # Get use_planning value from stored value
        use_planning = self._use_planning
        if use_planning is None:
            use_planning = state.get("use_planning", True)
        
        # Get use_explainer value from state
        use_explainer = self._use_explainer
        if use_explainer is None:
            use_explainer = state.get("use_explainer", True)
        
        update_state = {
            "messages": state.get("messages", []) + [tool_message],
            "agent_type": "data_exploration_agent",
            "routing_reason": f"Transferred to data exploration agent: {task_description}",
            "query": query,
            "plan": state.get("plan", ""),
            "steps": state.get("steps", []),
            "step_counter": state.get("step_counter", 0),
            "human_comment": state.get("human_comment"),
            "status": state.get("status", "approved"),
            "assistant_response": state.get("assistant_response", ""),
            "use_planning": use_planning,
            "use_explainer": use_explainer,
            "visualizations": state.get("visualizations", [])
        }
        
        return Command(
            goto="data_exploration_flow",
            update=update_state,
            graph=Command.PARENT,
        )
    
    self.transfer_to_data_exploration = transfer_to_data_exploration

# Add the method to the class
ExplainableAgent.create_handoff_tools = create_handoff_tools

print("✅ Handoff tools method added!")


In [ ]:
# Add the graph creation method
def create_graph(self):
    graph = StateGraph(ExplainableAgentState)
    
    # Add nodes
    graph.add_node("assistant", self.assistant_agent)  
    graph.add_node("data_exploration_flow", self.data_exploration_entry)  
    graph.add_node("planner", self.planner_node)
    graph.add_node("agent", self.agent_node)
    graph.add_node("tools", self.tools_node)
    graph.add_node("explain", self.explainer_node)
    graph.add_node("human_feedback", self.human_feedback)
    
    # Start with assistant for routing
    graph.set_entry_point("assistant")
    
    # Data exploration flow entry point - decides planning vs direct
    graph.add_conditional_edges(
        "data_exploration_flow",
        self.should_plan,
        {
            "planner": "planner",
            "agent": "agent"
        }
    )
    
    # Rest of the data exploration flow
    graph.add_edge("planner", "human_feedback")
    graph.add_conditional_edges(
        "human_feedback",
        self.should_execute,
        {
            "agent": "agent",
            "planner": "planner",
            "end": END
        }
    )
    graph.add_conditional_edges(
        "agent",
        self.should_continue,
        {
            "tools": "tools",
            "end": END
        }
    )
    graph.add_conditional_edges(
        "tools",
        self.should_explain,
        {
            "explain": "explain",
            "agent": "agent"
        }
    )
    graph.add_edge("explain", "agent")
    
    # Add memory checkpointer for interrupt functionality
    memory = self.mongo_memory
    return graph.compile(interrupt_before=["human_feedback"], checkpointer=memory)

# Add the method to the class
ExplainableAgent.create_graph = create_graph

print("✅ Graph creation method added!")


In [ ]:
# Add all the node methods to the ExplainableAgent class
def data_exploration_entry(self, state: ExplainableAgentState):
    return state

def should_plan(self, state: ExplainableAgentState):
    agent_type = state.get("agent_type", "data_exploration_agent")
    use_planning = state.get("use_planning", True)
    
    if agent_type == "data_exploration_agent":
        if use_planning:
            return "planner"  # Go through planning first
        else:
            return "agent"    # Go directly to data exploration

    # Default fallback to data exploration
    return "planner" if use_planning else "agent"

def human_feedback(self, state: ExplainableAgentState):
    pass

def should_execute(self, state: ExplainableAgentState):
    if state.get("status") == "approved":
        return "agent"
    elif state.get("status") == "feedback":
        return "planner"
    else:
        return "end"  # End the conversation

def planner_node(self, state: ExplainableAgentState):
    return self.planner.execute(state)

def should_continue(self, state: ExplainableAgentState):
    messages = state["messages"]
    last_message = messages[-1]
    
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        return "tools"
    else:
        return "end"  # End the conversation after agent completes

def should_explain(self, state: ExplainableAgentState):
    """Determine whether to use explainer node based on use_explainer flag"""
    use_explainer = state.get("use_explainer", True)
    
    if use_explainer:
        return "explain"
    else:
        return "agent"  # Skip explainer and go directly back to agent

# Add all methods to the class
ExplainableAgent.data_exploration_entry = data_exploration_entry
ExplainableAgent.should_plan = should_plan
ExplainableAgent.human_feedback = human_feedback
ExplainableAgent.should_execute = should_execute
ExplainableAgent.planner_node = planner_node
ExplainableAgent.should_continue = should_continue
ExplainableAgent.should_explain = should_explain

print("✅ Node methods added!")


In [ ]:
# Add the agent node method (the main reasoning node)
def agent_node(self, state: ExplainableAgentState):
    """Agent reasoning node"""
    messages = state["messages"]
    
    # Get tool descriptions dynamically
    tool_descriptions = "\n".join([f"- {tool.name}: {tool.description}" for tool in self.tools])
    prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")
    system_message = prompt_template.format(dialect="SQLite", top_k=5)
    # Add supported visualization types and variants to guide the LLM
    try:
        supported = get_supported_charts()
        charts_help = [
            f"- {chart_type}: variants = {', '.join(info.get('variants', []))}"
            for chart_type, info in supported.items()
        ]
        supported_charts = "\nSUPPORTED VISUALIZATIONS:\n" + "\n".join(charts_help) + "\n"
    except Exception:
        # Non-fatal if utils are unavailable
        pass
    system_message += f"""


You are a concise SQL database assistant. Answer only what is asked, nothing more.
- If user asks what tables exist, just list the tables
- If user asks for data, return the data in markdown table format when appropriate
- ONLY use smart_transform_for_viz tool when the user EXPLICITLY asks for a chart, graph, or visualization
- If user asks for multiple charts, call smart_transform_for_viz multiple times with different viz_type parameters, strictly only use supported types.
- Supported types: {supported_charts}
- If user did not specify a viz_type, decide the most appropriate type based on the data and context.

- Do NOT use smart_transform_for_viz for regular data queries - just return markdown tables
- Use tools only when necessary to answer the specific question
- NEVER generate images or base64 image data (no data:image/png;base64,...). Do not include markdown image tags for charts.
- For images referenced by the user (not charts) or urls from database, use markdown format: ![Alt text](image_url)
- For tabular data, format as markdown tables with proper headers and alignment
- For code, format as markdown code blocks with proper syntax highlighting
- Give explanation but be direct and brief - no unnecessary explanations or extra tool calls
- If you can't find any tables or columns, say so, stop the tool calling and provide information. Do not make up data.
- You can ask follow-up questions to clarify ambiguous requests or missing information.
- After providing requested data, you MAY briefly suggest ONE relevant next step if it adds clear value (e.g., "Would you like to visualize this data as a chart?"), but keep it minimal and unobtrusive.
- Do NOT repeatedly prompt for next actions or suggest multiple follow-ups.
- If the user's request is clear and complete, just answer it directly without suggestions.
- IMPORTANT: Do NOT call the same tool with same arguments multiple times. Call each tool only once per response.
- IMPORTANT: Only use smart_transform_for_viz tool when user specifically requests a visualization/chart
- IMPORTANT: Do not generate any images or base64 data for visualizations - only use the smart_transform_for_viz tool to return a JSON spec for the frontend renderer.
- If you accidentally produced an image or base64 output, remove it and instead produce a concise textual summary.
- Follow the plan strictly if one exists.

Examples (Visualization requests):
Example:
User: "Show a bar chart of the top 5 actors by film count"
Assistant (good):
1) Provide a one-paragraph summary of findings (no images)
2) Call smart_transform_for_viz with raw_data, columns, and viz_type='bar'
3) Just brief explanation

Bad Examples (Do NOT do):
- "![Chart](data:image/png;base64,...)"  (No base64 images)
- Calling visualization tool without explicit request
- Using unsupported chart types
"""

    # Bind tools to LLM so it can call them
    llm_with_tools = self.llm.bind_tools(self.tools)
    
    # Filter out previous system messages to avoid conflicts
    conversation_messages = [msg for msg in messages 
                           if not isinstance(msg, SystemMessage)]
    
    # Prepare messages for LLM
    all_messages = [SystemMessage(content=system_message)] + conversation_messages
    
    response = llm_with_tools.invoke(all_messages)
    
    return {
        "messages": messages + [response],
        "steps": state.get("steps", []),
        "step_counter": state.get("step_counter", 0),
        "query": state.get("query", ""),
        "plan": state.get("plan", "")
    }

# Add the method to the class
ExplainableAgent.agent_node = agent_node

print("✅ Agent node method added!")


In [ ]:
# Add the tools node and explainer node methods
def tools_node(self, state: ExplainableAgentState):
    messages = state["messages"]
    last_message = messages[-1]
    
    steps = state.get("steps", [])
    step_counter = state.get("step_counter", 0)

    # Execute tools
    tool_node = ToolNode(tools=self.tools)
    result = tool_node.invoke({"messages": messages})
    
    # Capture step information for explainer
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        for tool_call in last_message.tool_calls:
            step_counter += 1
            
            # Find the corresponding tool output
            tool_output = None
            for msg in result["messages"]:
                if hasattr(msg, 'tool_call_id') and msg.tool_call_id == tool_call['id']:
                    tool_output = msg.content
                    break
            
            step_info = {
                "id": step_counter,
                "type": tool_call['name'],
                "tool_name": tool_call['name'],
                "input": json.dumps(tool_call['args']),
                "output": tool_output or "No output captured",
                "context": state.get("query", "Database query"),
                "timestamp": datetime.now().isoformat()
            }
            
            # Add explanation fields with default values when explainer is disabled
            use_explainer = state.get("use_explainer", True)
            if not use_explainer:
                step_info.update({
                    "decision": f"Execute {tool_call['name']} tool",
                    "reasoning": f"Used {tool_call['name']} to process the query",
                    "confidence": 0.8,
                    "why_chosen": f"Selected {tool_call['name']} as the appropriate tool"
                })
            
            steps.append(step_info)
            
            if tool_call['name'] == "smart_transform_for_viz":
                try:
                    viz_dict = json.loads(tool_output)
                    state["visualizations"].append(viz_dict)
                except json.JSONDecodeError:
                    print(f"Failed to parse visualization output: {tool_output}")
                    state["visualizations"].append({"error": "Invalid JSON output"})

    return {
        "messages": result["messages"],
        "steps": steps,
        "step_counter": step_counter,
        "query": state.get("query", ""),
        "plan": state.get("plan", "")
    }

def explainer_node(self, state: ExplainableAgentState):
    """Explain the last step taken and ensure all steps have required fields"""
    steps = state.get("steps", [])
    updated_steps = []
    
    for i, step in enumerate(steps):
        # Create a copy to avoid mutating original state
        step_copy = step.copy()
        
        # Check if step is missing required fields
        missing_fields = [field for field in ["decision", "reasoning", "confidence", "why_chosen"] 
                         if field not in step_copy]
        
        if missing_fields:
            try:
                if i == len(steps) - 1:
                    # Get detailed explanation for the last step
                    explanation = self.explainer.explain_step(step_copy)
                    step_copy.update({
                        "decision": explanation.decision,
                        "reasoning": explanation.reasoning,
                        "why_chosen": explanation.why_chosen,
                        "confidence": explanation.confidence
                    })
                else:
                    # For previous steps, try to generate better defaults based on available data
                    tool_type = step_copy.get('type', 'unknown')
                    tool_result = step_copy.get('result', 'No result available')
                    
                    step_copy.update({
                        "decision": f"Execute {tool_type} tool",
                        "reasoning": f"Used {tool_type} to process the query. Result: {str(tool_result)[:100]}...",
                        "confidence": 0.7,  # Lower confidence for auto-generated explanations
                        "why_chosen": f"Selected {tool_type} as the appropriate tool for this step"
                    })
            except Exception as e:
                # Fallback if explanation generation fails
                step_copy.update({
                    "decision": f"Step {i+1} execution",
                    "reasoning": f"Error generating explanation: {str(e)}",
                    "confidence": 0.5,
                    "why_chosen": "Unable to determine reasoning"
                })
        
        updated_steps.append(step_copy)
    
    return {
        "messages": state["messages"],
        "steps": updated_steps,
        "step_counter": state.get("step_counter", 0),
        "query": state.get("query", ""),
        "plan": state.get("plan", "")
    }

# Add the methods to the class
ExplainableAgent.tools_node = tools_node
ExplainableAgent.explainer_node = explainer_node

print("✅ Tools and explainer node methods added!")


In [ ]:
# Add the control methods for state management
def get_interrupt_state(self, config=None):
    if config is None:
        config = {"configurable": {"thread_id": "main_thread"}}
    return self.graph.get_state(config)

def continue_with_feedback(self, user_feedback: str, status: str = "feedback", config=None):
    if config is None:
        config = {"configurable": {"thread_id": "main_thread"}}
    
    state_update = {"human_comment": user_feedback, "status": status}
    self.graph.update_state(config, state_update)
    
    events = list(self.graph.stream(None, config, stream_mode="values"))
    return events

def approve_and_continue(self, config=None):
    if config is None:
        config = {"configurable": {"thread_id": "main_thread"}}
    state_update = {"status": "approved"}
    self.graph.update_state(config, state_update)
    
    # Continue execution
    events = list(self.graph.stream(None, config, stream_mode="values"))
    return events

# Add the methods to the class
ExplainableAgent.get_interrupt_state = get_interrupt_state
ExplainableAgent.continue_with_feedback = continue_with_feedback
ExplainableAgent.approve_and_continue = approve_and_continue

print("✅ Control methods added!")


## Testing and Usage

Now let's create a simple testing interface to run the agent step by step and see state updates.


In [ ]:
# Initialize the agent
# You'll need to set your OpenAI API key
import os
os.environ["OPENAI_API_KEY"] = "your-api-key-here"  # Replace with your actual API key

# Initialize LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0.1)

# Set up database path (use one of the existing databases)
db_path = "../backend/src/resource/chinook.db"  # or art.db, sakila.db

# Create the agent
agent = ExplainableAgent(
    llm=llm,
    db_path=db_path,
    use_planning=True,  # Set to False to skip planning
    use_explainer=True  # Set to False to skip explanations
)

print("✅ Agent initialized successfully!")
print(f"Database: {db_path}")
print(f"Available tools: {[tool.name for tool in agent.tools]}")


In [ ]:
# Simple testing function to run the agent step by step
def test_agent_step_by_step(query: str, use_planning: bool = True, use_explainer: bool = True):
    """
    Test the agent with a query and show state updates step by step
    """
    print(f"🔍 Testing query: '{query}'")
    print(f"📋 Planning: {'Enabled' if use_planning else 'Disabled'}")
    print(f"🧠 Explainer: {'Enabled' if use_explainer else 'Disabled'}")
    print("-" * 50)
    
    # Create initial state
    initial_state = {
        "messages": [HumanMessage(content=query)],
        "query": query,
        "plan": "",
        "steps": [],
        "step_counter": 0,
        "human_comment": None,
        "status": "approved",
        "assistant_response": "",
        "use_planning": use_planning,
        "use_explainer": use_explainer,
        "response_type": None,
        "agent_type": "data_exploration_agent",
        "routing_reason": "",
        "visualizations": []
    }
    
    # Run the agent
    config = {"configurable": {"thread_id": "test_thread"}}
    
    try:
        # Stream the execution to see step-by-step updates
        events = list(agent.graph.stream(initial_state, config, stream_mode="values"))
        
        print(f"📊 Total events: {len(events)}")
        
        for i, event in enumerate(events):
            print(f"\n🔄 Event {i+1}:")
            print(f"   Status: {event.get('status', 'N/A')}")
            print(f"   Agent Type: {event.get('agent_type', 'N/A')}")
            print(f"   Steps: {len(event.get('steps', []))}")
            print(f"   Messages: {len(event.get('messages', []))}")
            
            # Show the last message if available
            if event.get('messages'):
                last_msg = event['messages'][-1]
                if hasattr(last_msg, 'content'):
                    content = last_msg.content[:100] + "..." if len(last_msg.content) > 100 else last_msg.content
                    print(f"   Last Message: {content}")
            
            # Show steps if any
            if event.get('steps'):
                print(f"   Latest Step: {event['steps'][-1].get('tool_name', 'N/A')}")
        
        # Show final result
        final_state = events[-1] if events else initial_state
        print(f"\n✅ Final Result:")
        print(f"   Total Steps: {len(final_state.get('steps', []))}")
        print(f"   Visualizations: {len(final_state.get('visualizations', []))}")
        
        return events, final_state
        
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return None, None

print("✅ Testing function ready!")


In [ ]:
# Example test cases
def run_test_cases():
    """Run various test cases to demonstrate the agent's capabilities"""
    
    test_queries = [
        "What tables are available in this database?",
        "Show me the top 5 artists by album count",
        "Create a bar chart of sales by country",
        "What is the total revenue from all sales?",
        "Show me a pie chart of genres distribution"
    ]
    
    print("🧪 Running Test Cases")
    print("=" * 60)
    
    for i, query in enumerate(test_queries, 1):
        print(f"\n📝 Test Case {i}: {query}")
        print("-" * 40)
        
        # Test with planning enabled
        events, final_state = test_agent_step_by_step(
            query=query,
            use_planning=True,
            use_explainer=True
        )
        
        if events:
            print(f"✅ Test {i} completed successfully")
        else:
            print(f"❌ Test {i} failed")
        
        print("\n" + "="*60)

# Uncomment to run test cases
# run_test_cases()

print("✅ Test cases ready! Uncomment run_test_cases() to execute them.")


In [ ]:
# Interactive testing - run a single query
def run_single_test():
    """Run a single test interactively"""
    
    # Example query - you can change this
    query = "What tables are available in this database?"
    
    print("🚀 Running Single Test")
    print("=" * 50)
    
    events, final_state = test_agent_step_by_step(
        query=query,
        use_planning=True,
        use_explainer=True
    )
    
    if events and final_state:
        print(f"\n📋 Detailed Results:")
        print(f"   Query: {final_state.get('query', 'N/A')}")
        print(f"   Plan: {final_state.get('plan', 'N/A')}")
        print(f"   Steps executed: {len(final_state.get('steps', []))}")
        
        # Show step details
        for i, step in enumerate(final_state.get('steps', []), 1):
            print(f"\n   Step {i}:")
            print(f"     Tool: {step.get('tool_name', 'N/A')}")
            print(f"     Decision: {step.get('decision', 'N/A')}")
            print(f"     Confidence: {step.get('confidence', 'N/A')}")
        
        # Show visualizations if any
        if final_state.get('visualizations'):
            print(f"\n   Visualizations: {len(final_state['visualizations'])}")
            for i, viz in enumerate(final_state['visualizations'], 1):
                print(f"     Viz {i}: {viz.get('chart_type', 'N/A')}")

# Uncomment to run a single test
# run_single_test()

print("✅ Single test function ready! Uncomment run_single_test() to execute.")
